In [1]:
'CQ2'.split('_')[-1]

'CQ2'

In [3]:
'cp_CQ2'.split('_')[-1]

'CQ2'

In [4]:
'phi_CQ2'.split('_')[-1]

'CQ2'

In [ ]:
# -------------------------------------------------- Penalty Calculation with Detailed Conditions --------------------------------------------------

# Define penalty variables for each time step
p_t = {t: model.addVar(vtype=GRB.CONTINUOUS, name=f"p_{t}") for t in Tset}

# Iterate through time steps except the last one
for t in Tset[:-1]:
    penalty = 0  # Initialize penalty
    for v in Vset:
        for j in Jset:
            if y[v, j, t] == 1: # only there is a task, we check the next task
                check_jump = False  # Used to track if the vessel move continue smoothly
                # Case 1: If task j is in the set of line tasks (Lset)
                if j in Lset:
                    l = j # the line is the j
                    tt = int(line_df[line_df['Line_No'] == l]['Time_underway_to_T'].iloc()[0])
                    end_time = t + cal_duration(tt)
                    t_p = end_time + 1 # Next task begin time 

                    for j_p in Jset:
                        if y[v,j,t] * y[v,j_p,t_p] == 1:# there is a imidiate following task
                            if j_p in Lset: # next task is line task
                                # Z_p[l,w,t] defines the line task end wharf
                                # z[w,l] defines the line task start wharf
                                if gp.quicksum(Z_p[l,w,end_time] * z[w,j_p] for w in Wset) <= 0: # start location of j' isn't the same as the end location of j
                                    check_jump = True
                            else: # j' in B/Bplus/Bc
                                w = j_p.split('_')[-1]
                                if Z_p[l,w,end_time] * y[v,j_p,t_p] == 0: # start location of j' isn't the same as the end location of j
                                    check_jump = True

                # Case 2: If the task is a waiting, charging, or crew pause task
                else:
                    end_time = (t + cal_duration(Dc) if j in Bc else
                                t)
                    t_p = end_time + 1 # Next task begin time 

                    for j_p in Jset:
                        if y[v,j,t] * y[v,j_p,t_p] == 1:# there is a imidiate following task
                            if j_p in Lset: # next task is line task
                                w = j.split('_')[-1]
                                # z[w,l] defines the line task start wharf
                                if y[v,j,t] * z[w,j_p] == 0: # start location of j' isn't the same as the end location of j
                                    check_jump = True
                            else: # j' in B/Bplus/Bc
                                if j.split('_')[-1] != j_p.split('_')[-1]:# start location of j' isn't the same as the end location of j
                                    check_jump = True
                if check_jump == True:
                    penalty += 1

    # Apply the constraint for penalties
    model.addConstr(p_t[t] >= penalty, name=f"penalty_{t}")


In [ ]:
# -------------------------------------------------- Penalty Calculation with Detailed Conditions --------------------------------------------------

# Define penalty variables for each time step
p_t = {t: model.addVar(vtype=GRB.CONTINUOUS, name=f"p_{t}") for t in Tset}

# Iterate through time steps except the last one
for t in Tset[:-1]:
    penalty_expr = []  # List to collect penalty terms

    for v in Vset:
        for j in Jset:
            # Only check if the vessel is assigned to a task at time t
            if y[v, j, t] == 1:
                check_jump = False  # Track if the vessel moves smoothly

                # Case 1: Task j is a line task
                if j in Lset:
                    l = j
                    tt = int(line_df[line_df['Line_No'] == l]['Time_underway_to_T'].iloc[0])
                    end_time = t + cal_duration(tt)
                    t_p = end_time + 1  # Next task begin time

                    for j_p in Jset:
                        if y[v, j_p, t_p] == 1:  # There is an immediate following task
                            if j_p in Lset:  # Next task is also a line task
                                if gp.quicksum(Z_prime[l, w, end_time] * z[w, j_p] for w in Wset) <= 0:
                                    check_jump = True
                            else:  # j_p belongs to B/Bplus/Bc
                                w = j_p.split('_')[-1]
                                if Z_prime[l, w, end_time] * y[v, j_p, t_p] == 0:
                                    check_jump = True

                # Case 2: Task j is a waiting, charging, or crew pause task
                else:
                    end_time = t + cal_duration(Dc) if j in Bc else t
                    t_p = end_time + 1  # Next task begin time

                    for j_p in Jset:
                        if y[v, j_p, t_p] == 1:  # There is an immediate following task
                            if j_p in Lset:  # Next task is a line task
                                w = j.split('_')[-1]
                                if y[v, j, t] * z[w, j_p] == 0:
                                    check_jump = True
                            else:  # j_p belongs to B/Bplus/Bc
                                if j.split('_')[-1] != j_p.split('_')[-1]:  # Different starting location
                                    check_jump = True

                # If a discontinuity is detected, add it to the penalty expression
                if check_jump:
                    penalty_expr.append(1)

    # Apply the penalty constraint
    model.addConstr(p_t[t] >= gp.quicksum(penalty_expr), name=f"penalty_{t}")

# Define total penalty and update rebalancing time
total_penalty = gp.quicksum(p_t[t] for t in Tset)
rebalancing_time += total_penalty


In [ ]:
import gurobipy as gp
from gurobipy import GRB


total_jumps = 0
for v in Vset:
    for t in Tset[:-1]:
        # Find which wharf (if any) vessel v is assigned at time t.
        wharf_t = None
        for w in Wset:
            if u[v, w, t].X > 0:
                wharf_t = w
                break  # assuming at most one is 1
        
        # Find which wharf (if any) vessel v is assigned at time t+1.
        wharf_tp1 = None
        for w in Wset:
            if u[v, w, t+1].X > 0:
                wharf_tp1 = w
                break
        
        # Count a jump if the vessel is assigned at both times and the wharf changes.
        if wharf_t is not None and wharf_tp1 is not None and wharf_t != wharf_tp1:
            total_jumps += 1

print("Total jump count:", total_jumps)

